In [ ]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets
import torch.nn.functional as f
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
import pandas as pd
import math

In [ ]:
import h5py
import numpy as np
filepath = 'drive/MyDrive/DeepLearning_Project/Data/Dic_ocean.mat'
arrays = {}
f = h5py.File(filepath)
for k, v in f.items():
    arrays[k] = np.array(v)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  """


In [ ]:
dataset = arrays['Dic_ocean']

In [ ]:
df = pd.DataFrame(data=dataset)

In [ ]:
df = df[[0, 1, 2, 3, 4, 5,6,7,8,9,10,11,12,18]]

In [ ]:
df.shape

(5406505, 14)

In [ ]:
df.mean(axis=0)

0     174.211579
1     105.244170
2     193.862524
3     132.074897
4     214.917496
5     216.924162
6     162.944387
7     252.803301
8     222.612377
9     266.225172
10    259.815200
11    255.941583
12    264.039179
18      0.018301
dtype: float64

In [ ]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,18
count,5.406505e+06,5.406505e+06,5.406505e+06,5.406505e+06,5.406505e+06,5.406505e+06,5.406505e+06,5.406505e+06,5.406505e+06,5.406505e+06,5.406505e+06,5.406505e+06,5.406505e+06,2.857203e+06
mean,1.742116e+02,1.052442e+02,1.938625e+02,1.320749e+02,2.149175e+02,2.169242e+02,1.629444e+02,2.528033e+02,2.226124e+02,2.662252e+02,2.598152e+02,2.559416e+02,2.640392e+02,1.830058e-02
std,2.434191e+01,3.780310e+01,2.049592e+01,3.367047e+01,2.104154e+01,1.379414e+01,2.746119e+01,1.457784e+01,2.579099e+01,1.509740e+01,1.992599e+01,8.348644e+00,1.048605e+01,1.198194e-01
min,1.547800e+02,7.909000e+01,1.500700e+02,9.224000e+01,1.537000e+02,1.438800e+02,1.224700e+02,7.707000e+01,7.630000e+01,8.828000e+01,8.635000e+01,9.587000e+01,9.142000e+01,0.000000e+00
25%,1.619100e+02,8.864000e+01,1.801500e+02,1.109200e+02,1.968200e+02,2.068500e+02,1.440200e+02,2.416600e+02,2.015800e+02,2.574600e+02,2.460100e+02,2.499100e+02,2.581000e+02,0.000000e+00
50%,1.663600e+02,9.249000e+01,1.866400e+02,1.206500e+02,2.095300e+02,2.126400e+02,1.546600e+02,2.507600e+02,2.213900e+02,2.673900e+02,2.626100e+02,2.544000e+02,2.638200e+02,0.000000e+00
75%,1.723600e+02,9.738000e+01,1.996600e+02,1.388100e+02,2.312000e+02,2.227200e+02,1.717100e+02,2.638600e+02,2.427200e+02,2.782800e+02,2.771800e+02,2.610800e+02,2.710900e+02,0.000000e+00
max,2.788000e+02,2.631800e+02,2.817200e+02,2.794700e+02,2.860600e+02,2.818400e+02,2.785900e+02,2.975900e+02,2.915400e+02,2.995200e+02,2.991500e+02,2.876400e+02,2.956100e+02,1.160687e+01


In [ ]:
count_nan = len(df) - df.count()

In [ ]:
count_nan

0           0
1           0
2           0
3           0
4           0
5           0
6           0
7           0
8           0
9           0
10          0
11          0
12          0
18    2549302
dtype: int64

In [ ]:
df = df[pd.notnull(df[18])]

In [ ]:
np.count_nonzero(df, axis=0)

array([2857203, 2857203, 2857203, 2857203, 2857203, 2857203, 2857203,
       2857203, 2857203, 2857203, 2857203, 2857203, 2857203,  226088])

In [ ]:
test_split = .3
random_seed =1

In [ ]:
data1 = df.to_numpy()

In [ ]:
data1.shape

(2857203, 14)

In [ ]:
Full_data = data1.astype(np.float)

In [ ]:
from scipy import stats

In [ ]:
from torch.utils.data.sampler import SubsetRandomSampler
validation_split = .3
dataset_size = data1.shape[0]
indices = list(range(dataset_size))
split = int(np.floor(test_split * dataset_size))
np.random.seed(random_seed)
np.random.shuffle(indices)
train_indices, test_indices = indices[split:], indices[:split]

# Creating Data Samples:
train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

In [ ]:
# setup hyperparameters
epochs = 200
lr = 1e-4
batchsize = 1024 #16384 #1024

In [ ]:
train_loader = torch.utils.data.DataLoader(Full_data,sampler=train_sampler, batch_size=batchsize)
test_loader = torch.utils.data.DataLoader(Full_data,sampler=test_sampler, batch_size=batchsize)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

In [ ]:
# a)setting up neural net
class neural_net(nn.Module):
  def __init__(self):
      super(neural_net, self).__init__()
      self.fc1 = nn.Linear(13,256) # 26 feature
      self.fc2 = nn.Linear(256,128)
      self.fc3 = nn.Linear(128, 1)
      self.relu = nn.ReLU()
      self.droput = nn.Dropout(0.1)
  def encode(self, x):
      x = F.relu(self.fc1(x))
      mu = self.fc2_mu(x)
      sigma = self.fc2_sigma(x)
      return mu, sigma

  def reparameterize(self, mu, sigma):
      std = torch.exp(0.5*sigma) # standard deviation ( sigma = log((std)**2) )
      noise = torch.randn_like(std)
      z = mu + std*noise # The "source" to the decoder and we want to make this to unit Gaussian after training.
      return z


  def forward(self, x):
      z= self.encode(X[-1])
      
      x = self.fc1(x) #
      x = self.relu(x) #25   
      x = self.droput(x)        
      x = torch.flatten(x, start_dim=1)
      x = self.relu(self.fc2(x))
      #x = self.droput(x) 
      x = self.fc3(x)

      return x

In [ ]:
neural = neural_net()
neural.to(device)

neural_net(
  (fc1): Linear(in_features=13, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
  (droput): Dropout(p=0.1, inplace=False)
)

In [ ]:
dataiter = iter(train_loader)
array = dataiter.next()

print(array.shape)

torch.Size([1024, 14])


In [ ]:
it = iter(train_loader)
Input_batch = next(it)
#print(Input_batch[:,13])
Input_batch = Input_batch[:,0:13].float()
Input_batch = Input_batch.to(device)
print(neural.forward(Input_batch).shape)

torch.Size([1024, 1])


In [ ]:
len(train_loader)

1954

In [ ]:
def fit(model,trainloader):
  Loss = nn.MSELoss()
  optimizer = optim.AdamW(model.parameters(), lr)
  #d-dimensional vector representing diag(Gt) to store a running total of the squares of the gradients.
  for e in range(epochs):
      running_loss = 0
      for input in trainloader:  
          input = input.to(device)    
          optimizer.zero_grad()         
          output = model(input[:,0:13].float())
          #print(output)
          #output = output.view(-1,1,1,1)
          loss = Loss(output, input[:,13].float().view(-1,1))
          #print(loss)
          loss.backward()
          optimizer.step()
          running_loss += loss.item()
      print(math.sqrt(running_loss/len(trainloader)))
      if(math.sqrt(running_loss/len(trainloader))<0.09):
        break
          #print("Epoch {} - Training loss: {} -Test Accuracy:{}".format(e, running_loss))

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
def evaluate_model(model, test_loader):
    predictions, actuals = list(), list()
    model.evaluate()
    for inputs in test_loader:
        # evaluate the model on the test set
        inputs = inputs.to(device)
        yhat = model(inputs[:,0:13].float())
        # retrieve numpy array
        yhat = yhat.cpu().detach().numpy()
        actual = inputs[:,13].cpu().float().numpy()
        actual = actual.reshape((len(actual), 1))
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = np.vstack(predictions), np.vstack(actuals)
    # calculate mse
    mse = mean_squared_error(actuals, predictions)
    rmse = math.sqrt(mse)
    print(mse,rmse)
    return mse,rmse

In [ ]:
fit(neural,train_loader)

1.4579067236033105
0.16607465915068145
0.16444538977182352
0.15825766172664435
0.1514448343423
0.1414346954931934
0.13156862323099525
0.12218548124863562
0.11742176912375517
0.1144888671983654
0.10835625526872125
0.10600304841507051
0.10495091038888982
0.1040057501000664
0.10318297367017777
0.10241182041823563
0.10170293464954723
0.10108656268318163
0.1002819185760915
0.09949790146526276
0.0990004125273284
0.09827360388905235
0.09797869037289088
0.09762245326354332
0.09744980346700308
0.09719101059192987
0.09696201537677168
0.09683074421124759
0.09658518801428843
0.09631118629627067
0.09616053832562299
0.09593366605121402
0.09586953048051579
0.09576343513045038
0.09539542618965861
0.0953160770880247
0.09536649232680967
0.09507651590059077
0.09504647588353228
0.09497578463346384
0.09471901822228959
0.09470019496168866
0.09462822732161534
0.09448011851809746
0.09445397784157164
0.09430581615854329
0.09432581725767658
0.094193207166615
0.09420939825538219
0.0941937103411705
0.094091575872

In [ ]:
torch.save(neural.state_dict(), "drive/MyDrive/DeepLearning_Project/Models/Adamw200lr10e-41024batch")

In [ ]:
evaluate_model(neural,test_loader)

0.008273844 0.0909606710389266


(0.008273844, 0.0909606710389266)